In [18]:
import numpy as np
import scipy as sp
import sympy as sm
import pandas as pd

import bottleneck as bn
import numexpr as ne
import numba as nb

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


#sns.set()

In [19]:

import torch

from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

torch.__version__

'1.7.0'

In [20]:
train_dataloader = torch.utils.data.DataLoader(datasets.CIFAR10('../cifar',
                                                                download=True,
                                                                train=True,
                                                                transform=transforms.Compose([
                                                                    transforms.ToTensor(),
                                                                ])),
                                               batch_size=64,
                                               shuffle=True)

test_dataloader = torch.utils.data.DataLoader(datasets.CIFAR10('../cifar',
                                                               download=True,
                                                               train=False,
                                                               transform=transforms.Compose([
                                                                   transforms.ToTensor(),

                                                               ])),
                                              batch_size=64,
                                              shuffle=True)


loss: 1.626184  [32000/50000]
Files already downloaded and verified
Files already downloaded and verified


In [21]:
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break


Shape of X [N, C, H, W]:  torch.Size([64, 3, 32, 32])
Shape of y:  torch.Size([64]) torch.int64


In [22]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


class NeuralNetwork1(nn.Module):
    def __init__(self):
        super(NeuralNetwork1, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(32 * 32 * 3, 10),
            nn.Softmax(-1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


class NeuralNetwork2(nn.Module):
    def __init__(self):
        super(NeuralNetwork2, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64 * 64, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
            nn.Dropout(p=0.5),
            # nn.ReLU(),
            nn.Softmax(-1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


class NeuralNetworkCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(128 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.dropout(x, p=0.3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Using cuda device


In [23]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn, title='Test'):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"{title} Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

Однослойная сеть, разделение выборки случайно

In [25]:


!pip install ray[tune]
!pip uninstall -y dataclasses

loss: 1.770653  [38400/50000]
loss: 1.633596  [44800/50000]
Train Error: 
 Accuracy: 60.5%, Avg loss: 0.017476 

Test Error: 
 Accuracy: 58.4%, Avg loss: 0.018388 



In [26]:
import ray
from ray import tune


def my_func(config, reporter):
    device = config['device']
    model = NeuralNetworkCNN().to(device)
    print(model)
    loss_fn = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    epochs = 10
    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(train_dataloader, model, loss_fn, title='Train')
        acc = test(test_dataloader, model,loss_fn, title='Test')
        reporter(mean_accuracy=acc)
    print("Done!")


ray.init(local_mode=True, ignore_reinit_error=True)
e = tune.run(my_func,
             resources_per_trial={"gpu": 1},
             metric='mean_accuracy',
             stop={"mean_accuracy": 0.95},
             config={
                 "lr": tune.loguniform(1e-4, 1e-2),
                 "device": "cuda"
             },
             num_samples=4
             )

2021-06-17 20:12:01,324	INFO worker.py:665 -- Calling ray.init() again after it has already been called.


NeuralNetworkCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)
Epoch 1
-------------------------------
loss: 2.315755  [    0/50000]
loss: 2.115036  [ 6400/50000]
Train Error: 
 Accuracy: 34.6%, Avg loss: 0.027089 

loss: 1.670417  [12800/50000]
Test Error: 
 Accuracy: 34.3%, Avg loss: 0.027144 

loss: 1.973652  [19200/50000]
loss: 1.632050  [25600/50000]
loss: 1.678942  [32000/50000]
loss: 1.628077  [38400/50000]
loss: 1.427530  [44800/50000]
Train Error: 
 Accuracy: 45.6%, Avg loss: 0.022962 



2021-06-17 20:12:29,685	WARNING util.py:152 -- The `start_trial` operation took 23.782 s, which may be a performance bottleneck.


Test Error: 
 Accuracy: 46.1%, Avg loss: 0.023041 



Trial name,status,loc,lr
my_func_486c8_00000,RUNNING,,0.00144876


...

Result for my_func_486c8_00003:
  date: 2021-06-17_20-21-39
  done: false
  experiment_id: e23e828eb8284596bacd2479914b9813
  hostname: 13a89f6488bf
  iterations_since_restore: 9
  mean_accuracy: 0.4632
  node_ip: 172.19.2.2
  pid: 164
  time_since_restore: 127.56245970726013
  time_this_iter_s: 14.23304533958435
  time_total_s: 127.56245970726013
  timestamp: 1623961299
  timesteps_since_restore: 0
  training_iteration: 9
  trial_id: 486c8_00003
  
Epoch 10
-------------------------------
loss: 1.442043  [    0/50000]
loss: 1.502857  [ 6400/50000]
loss: 1.545196  [12800/50000]
loss: 1.649372  [19200/50000]
loss: 1.553633  [25600/50000]
loss: 1.707816  [32000/50000]
loss: 1.566952  [38400/50000]
loss: 1.472685  [44800/50000]
Train Error: 
 Accuracy: 47.9%, Avg loss: 0.022699 

Test Error: 
 Accuracy: 47.0%, Avg loss: 0.023182 



Trial name,status,loc,lr,acc,iter,total time (s)
my_func_486c8_00003,RUNNING,172.19.2.2:164,0.00855231,0.4632,9,127.562
my_func_486c8_00000,TERMINATED,,0.00144876,0.6552,10,149.484
my_func_486c8_00001,TERMINATED,,0.00015086,0.5694,10,143.338
my_func_486c8_00002,TERMINATED,,0.000133983,0.5565,10,143.134


Result for my_func_486c8_00003:
  date: 2021-06-17_20-21-52
  done: false
  experiment_id: e23e828eb8284596bacd2479914b9813
  hostname: 13a89f6488bf
  iterations_since_restore: 10
  mean_accuracy: 0.4702
  node_ip: 172.19.2.2
  pid: 164
  time_since_restore: 141.39892578125
  time_this_iter_s: 13.836466073989868
  time_total_s: 141.39892578125
  timestamp: 1623961312
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 486c8_00003
  
Done!
Result for my_func_486c8_00003:
  date: 2021-06-17_20-21-52
  done: true
  experiment_id: e23e828eb8284596bacd2479914b9813
  experiment_tag: 3_lr=0.0085523
  hostname: 13a89f6488bf
  iterations_since_restore: 10
  mean_accuracy: 0.4702
  node_ip: 172.19.2.2
  pid: 164
  time_since_restore: 141.39892578125
  time_this_iter_s: 13.836466073989868
  time_total_s: 141.39892578125
  timestamp: 1623961312
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 486c8_00003
  


Trial name,status,loc,lr,acc,iter,total time (s)
my_func_486c8_00000,TERMINATED,,0.00144876,0.6552,10,149.484
my_func_486c8_00001,TERMINATED,,0.00015086,0.5694,10,143.338
my_func_486c8_00002,TERMINATED,,0.000133983,0.5565,10,143.134
my_func_486c8_00003,TERMINATED,,0.00855231,0.4702,10,141.399


2021-06-17 20:21:52,866	INFO tune.py:450 -- Total run time: 591.54 seconds (588.71 seconds for the tuning loop).


In [38]:
df=e.dataframe()
print('best config')
df[df['mean_accuracy']==df['mean_accuracy'].max()][['mean_accuracy','config/lr']]


best config


,mean_accuracy,config/lr
0,0.6552,0.001449
